In [ ]:
import os
import shutil
import warnings

import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

from train import create_net
from utils import transforms as MT
from utils.datasets import AortaDataset3DCenter

warnings.filterwarnings("ignore")
np.random.seed(63910)
torch.manual_seed(53152)
torch.cuda.manual_seed_all(7987)
torch.backends.cudnn.deterministic = True

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
n_channels, n_classes, batch_size = 1, 4, 128

transform = T.Compose([
    MT.Resize3D(81),
    MT.CenterCrop3D(81),
    MT.ToTensor3D(),
])

train = AortaDataset3DCenter('/nfs3-p1/zsxm/dataset/aorta_classify_ct_-100_500/center/train/', transform=transform, depth=7)
val = AortaDataset3DCenter('/nfs3-p1/zsxm/dataset/aorta_classify_ct_-100_500/center/val/', transform=transform, depth=7)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=False)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=False)

In [ ]:
n_channels, n_classes, batch_size = 1, 4, 128

transform = T.Compose([
    T.Resize(81), # 缩放图片(Image)，保持长宽比不变，最短边为img_size像素
    T.CenterCrop(81), # 从图片中间切出img_size*img_size的图片
    T.ToTensor(), # 将图片(Image)转成Tensor，归一化至[0, 1]
])

train = ImageFolder('/nfs3-p1/zsxm/dataset/aorta_classify_ct_-100_500/train/', transform=transform, loader=lambda path: Image.open(path))
val = ImageFolder('/nfs3-p1/zsxm/dataset/aorta_classify_ct_-100_500/val/', transform=transform, loader=lambda path: Image.open(path))
train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=False)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=False)

In [ ]:
net = create_net(device, 'resnet', 34, n_channels, n_classes, '/nfs3-p2/zsxm/temp_path/single81.pth', entire=True, flag_3d=False)
net.eval()
print('')

In [ ]:
class HookModule:
    def __init__(self, model, module):
        self.model = model
        self.handle = module.register_forward_hook(self._get_output)
        
    def _get_output(self, module, inputs, outputs):
        self.outputs = outputs
    
    def grads(self, outputs, retain_graph=True, create_graph=True):
        grads = torch.autograd.grad(outputs=outputs, inputs=self.outputs, retain_graph=retain_graph, create_graph=create_graph)
        self.model.zero_grad()
        print(grads[0].shape)
        return grads[0]
    
    def remove(self):
        self.handle.remove()
        
hook = HookModule(net, net.encoder.layer2[3].conv2)

In [ ]:
hook.remove()

In [ ]:
def get_grads(dataset, dataloader, end=500, start=0):
    idx = 0
    score_list = [[] for _ in range(n_classes)]
    err_list = [[] for _ in range(n_classes)]
    for imgs, true_categories in tqdm(dataloader, total=len(dataloader), desc='Dataset', unit='batch', leave=False):
        imgs = imgs.to(device=device, dtype=torch.float32)
        true_categories = true_categories.to(device=device, dtype=torch.long)

        with torch.no_grad():
            categories_pred = torch.softmax(net(imgs), dim=1)
            labels_pred = categories_pred.argmax(dim=1)
            for i in range(len(true_categories)):
                if labels_pred[i] == true_categories[i]:
                    score_list[true_categories[i].item()].append((categories_pred[i, labels_pred[i]].item(), idx))
                else:
                    #err_list[true_categories[i].item()].append(dataset.samples[idx][0])
                    pass
                idx += 1
    
    grad_list = []
    mask_list = []
    for i in range(n_classes):
        print(len(score_list[i]))
        score_list[i].sort(key=lambda x: x[0], reverse=True)
        print(score_list[i][end-1])
        idx_list = [score[1] for score in score_list[i][start: end]]
        tensor_list = []
        for idx in idx_list:
            tensor_list.append(dataset[idx][0])
        batch = torch.stack(tensor_list, dim=0)
        pred = net(batch.to(device))
        nll_loss = F.nll_loss(pred, (torch.ones(batch.size(0))*i).to(device, dtype=torch.long))
        grads = hook.grads(-nll_loss, True, False)
        nll_loss.backward()
        #grads = grads.abs().sum(dim=(2,3,4)).mean(dim=0)
        grads = F.relu(grads).sum(dim=(2,3)).mean(dim=0)
        print(grads.shape)
        grad_list.append(grads.cpu().numpy())
        mask = (grads>grads.mean()).long()
        mask_list.append(mask.cpu().numpy())

    grads = np.array(grad_list)
    masks = np.array(mask_list)
    return grads, masks, err_list

In [ ]:
grads, masks, err_list = get_grads(train, train_loader, 500)

In [ ]:
# for i in range(n_classes):
#     print(len(err_list[i]))
#     outpath = os.path.join('/nfs3-p2/zsxm/dataset/aorta_classify_ct_-100_500/err_imgs', str(i))
#     os.makedirs(outpath, exist_ok=True)
#     for img in err_list[i]:
#         shutil.copy(img, os.path.join(outpath, img.split('/')[-1]))

In [ ]:
hm = grads - np.min(grads)
hm = hm / np.max(hm)
print(hm)

def view_grads(grads, fig_w, fig_h, fig_path='./heatmap.png'):
    f, ax = plt.subplots(figsize=(fig_w, fig_h), ncols=1)
    ax.set_xlabel('convolutional kernel')
    ax.set_ylabel('category')
    sns.heatmap(grads, annot=False, ax=ax)
    plt.savefig(fig_path, bbox_inches='tight')
    plt.show()
    #plt.clf()
    
view_grads(hm, 30, 5)

In [ ]:
view_grads(masks, 30, 5)

In [ ]:
np.save('/nfs3-p2/zsxm/temp_path/pos_layer2_conv2.npy', masks)

In [ ]:
a = (1,2,3)
x, y, z = a
print(x, y, z)

In [ ]:
class AdNoise:
    def __init__(self, model, module):
        self.model = model
        self.handle = module.register_forward_hook(self._get_output)
        
    def _get_output(self, module, inputs, outputs):
        self.outputs = outputs
        outputs += torch.randn(outputs.shape).to(outputs.device)
    
    def grads(self, outputs, retain_graph=True, create_graph=True):
        grads = torch.autograd.grad(outputs=outputs, inputs=self.outputs, retain_graph=retain_graph, create_graph=create_graph)
        self.model.zero_grad()
        return grads[0]
    
    def remove(self):
        self.handle.remove()
        
class Mul2:
    def __init__(self, model, module):
        self.model = model
        self.handle = module.register_forward_hook(self._get_output)
        
    def _get_output(self, module, inputs, outputs):
        self.outputs = outputs
        outputs *= 2
    
    def grads(self, outputs, retain_graph=True, create_graph=True):
        grads = torch.autograd.grad(outputs=outputs, inputs=self.outputs, retain_graph=retain_graph, create_graph=create_graph)
        self.model.zero_grad()
        return grads[0]
    
    def remove(self):
        self.handle.remove()

In [ ]:
class Test(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 3)
        self.conv2 = nn.Conv2d(10, 1, 3)
        nn.init.constant_(self.conv1.weight, 1)
        nn.init.constant_(self.conv1.weight, 0)
        
    def forward(self, x):
        x = self.conv1(x)
        x = torch.exp(x)
        x = self.conv2(x)
        return x
    
test = Test()

In [ ]:
h2 = Mul2(test, test.conv1)

In [ ]:
h2.remove()

In [ ]:
hn = AdNoise(test, test.conv1)

In [ ]:
hn.remove()

In [ ]:
a = torch.ones(2,3,6,6)
res = test(a)
ressum = res.sum()
print(res)
print(ressum)

In [ ]:
print(h2.outputs)
print(h2.grads(ressum))

In [ ]:
print(h2.outputs)
print(h2.grads(ressum))

In [ ]:
print(hn.outputs)
print(hn.grads(ressum))

In [ ]:
t = torch.ones(2,2)
print(id(t))

In [ ]:
t += 1
print(id(t))

In [ ]:
t = t+1
print(id(t))

In [ ]:
a = 1
b = a
print(id(a), id(b))

In [ ]:
def bad_append(new_item, a_list=[]):
 
    a_list.append(new_item)
 
    return a_list

In [ ]:
print(bad_append(1))

In [ ]:
a = (1,2)
a[0] = 3

In [ ]:
class Test():
    a = None
    
def fun(x):
    x = Test()
    x.a = 7

In [ ]:
t, k = Test(), Test()
t.a = 1
k.a = 2
p = t
print(p.a, t.a, k.a)
print(id(p), id(t), id(k))
p = k
print(p.a, t.a, k.a)
print(id(p), id(t), id(k))